# Basic use example
## Create a table using zonalStats function and export it as a csv file to Google Drive

#### Load modules

In [4]:
import ee
import s2_gee_toolbox as s2g

from general_gee_toolbox import zonalStats
from general_gee_toolbox import showTaskManager

ee.Initialize()

#### Generate a featureCollection from a list of points or load a custom one 

In [5]:

points = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-0.66522150, 39.93055864]), {'id': 1}),
  ee.Feature(ee.Geometry.Point([-0.72310899, 39.98148865]), {'id': 2}),
  ee.Feature(ee.Geometry.Point([-0.60845867, 39.07450846]), {'id': 3}),
  ee.Feature(ee.Geometry.Point([-1.67793037, 38.89151445]), {'id': 4}),
  ee.Feature(ee.Geometry.Point([-1.84322937, 38.90086940]), {'id': 5})
])

In [6]:
i_date = '2019-01-01'
e_date = '2020-01-01'

# Get Sentinel-2 Cloudless Collection and add some spectral indices
cloud_free_s2 = s2g.get_cloudless_col(points, i_date, e_date).map(s2g.add_NDVI).map(s2g.add_DVI).map(s2g.add_NDWI).map(s2g.add_DWI).map(s2g.add_GNDVI)
era5_coll = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filter(ee.Filter.date(i_date, e_date)).map(lambda image: image.clip(points))

# Generate parmeters for the zonalStats function
z_params_S2 = {
  'reducer': 'full',
  'scale': 10, 
  'crs': 'EPSG:4326', 
  'bands': ['ndvi', 'dvi', 'ndwi', 'dwi', 'gndvi', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'], 
  'bandsRename': ['NDVI', 'DVI', 'NDWI', 'DWI', 'GNDVI', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'],
  'datetimeName': 'date',
  'datetimeFormat': 'YYYY-MM-dd',
  'imgProps': ['GRANULE_ID']
}

z_params_era5 = {
  'reducer': ee.Reducer.mean(),
  'scale': 10, # Must be 10 in order to conserve all polygons
  'crs': 'EPSG:4326',
  'bands': ['total_precipitation', 'temperature_2m', 'soil_temperature_level_1', 'u_component_of_wind_10m', 'v_component_of_wind_10m'], 
  'bandsRename': None,
  'datetimeName': 'date',
  'datetimeFormat': 'YYYY-MM-dd',
}

zonalS2 = zonalStats(cloud_free_s2, points, **z_params_S2)
zonalERA5 = zonalStats(era5_coll, points, **z_params_era5)

# Export featureCollection as csv files to Goolgle Drive 
taskS2_name  = 'S2_TimeSeries'
taskS2 = ee.batch.Export.table.toDrive(
    collection = zonalS2,
    folder = 'earthengine',
    description = taskS2_name,
    fileFormat = 'CSV'
  )

taskERA5_name = 'ERA5_TimeSeries'
taskERA5 = ee.batch.Export.table.toDrive(
    collection = zonalERA5,
    folder =  'earthengine',
    description = taskERA5_name,
    fileFormat = 'CSV'
  )

taskS2.start()
taskERA5.start()

showTaskManager(taskS2, taskERA5)

Output()